In [2]:
%matplotlib inline 
from datetime import datetime 
from random import randrange 
import random
import operator 
import pandas as pd 
import matplotlib.pyplot as plt 
import re 
import numpy as np
import collections
import time

In [3]:
df = pd.read_csv('capture20110811.pcap.netflow.labeled', sep ='\s+')
df.head()

,#Date_flow,start,Durat,Prot,Src_IP_Addr:Port,->,Dst_IP_Addr:Port,Flags,Tos,Packets,Bytes,Flows,Label(LEGITIMATE:Botnet:Background)
0,2011-08-11,10:10:00.003,0.967,UDP,89.31.8.11:23929,->,147.32.84.229:13363,INT,0,2,135,1,Background
1,2011-08-11,10:10:00.003,0.967,UDP,147.32.84.229:13363,->,89.31.8.11:23929,INT,0,2,276,1,Background
2,2011-08-11,10:10:00.006,0.000,UDP,208.88.186.6:34042,->,147.32.84.229:13363,INT,0,1,62,1,Background
3,2011-08-11,10:10:00.008,0.000,UDP,92.118.218.77:55246,->,147.32.84.229:13363,INT,0,1,78,1,Background
4,2011-08-11,10:10:00.009,0.000,UDP,182.185.139.181:10223,->,147.32.84.229:13363,INT,0,1,72,1,Background


In [4]:
df[['Src_IP_Addr', 'Src_Port']] = df['Src_IP_Addr:Port'].str.split(':', n=1, expand=True)
df[['Dst_IP_Addr', 'Dst_Port']] = df['Dst_IP_Addr:Port'].str.split(':', n=1, expand=True)
#df['Date']= df['#Date_flow'].map(str)+ '' + df['start']
#df['Date']= pd.to_datetime(df['Date'], format = '%Y-%m-%d %H:%M:%S.%f', exact=True)
df['Label'] = df['Label(LEGITIMATE:Botnet:Background)']

df = df.drop(labels=['->', 'Label(LEGITIMATE:Botnet:Background)', 'Src_IP_Addr:Port', 'Dst_IP_Addr:Port'], axis=1)

print(df.shape)
print(df.columns)
df.head()

(6351188, 14)
Index(['#Date_flow', 'start', 'Durat', 'Prot', 'Flags', 'Tos', 'Packets',
       'Bytes', 'Flows', 'Src_IP_Addr', 'Src_Port', 'Dst_IP_Addr', 'Dst_Port',
       'Label'],
      dtype='object')


,#Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Src_IP_Addr,Src_Port,Dst_IP_Addr,Dst_Port,Label
0,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,135,1,89.31.8.11,23929,147.32.84.229,13363,Background
1,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,276,1,147.32.84.229,13363,89.31.8.11,23929,Background
2,2011-08-11,10:10:00.006,0.000,UDP,INT,0,1,62,1,208.88.186.6,34042,147.32.84.229,13363,Background
3,2011-08-11,10:10:00.008,0.000,UDP,INT,0,1,78,1,92.118.218.77,55246,147.32.84.229,13363,Background
4,2011-08-11,10:10:00.009,0.000,UDP,INT,0,1,72,1,182.185.139.181,10223,147.32.84.229,13363,Background


In [5]:
print(df['Src_IP_Addr'].value_counts()[0:10])
infected_IP = '147.32.84.165'
val = df['Src_IP_Addr'].value_counts()[infected_IP] + df['Dst_IP_Addr'].value_counts()[infected_IP]
print("")
print("Infected ip occurrences: ", val)
# top 10 freq


147.32.84.229    1458791
147.32.80.9       573079
147.32.84.59      347413
147.32.84.138     331209
147.32.86.165      82589
147.32.84.165      45261
147.32.84.171      34186
147.32.84.118      34027
147.32.85.25       31279
147.32.86.20       28073
Name: Src_IP_Addr, dtype: int64

Infected ip occurrences:  54433


In [82]:
inf = df.loc[(df['Src_IP_Addr']== infected_IP)]
dst_hosts = inf['Dst_IP_Addr'].values
unique_dst_hosts = np.unique(dst_hosts)

print("Number of hosts connected with infected: ", len(unique_dst_hosts))
print(inf.shape)
inf.head()

Number of hosts connected with infected:  1662
(45261, 14)


,#Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Src_IP_Addr,Src_Port,Dst_IP_Addr,Dst_Port,Label
540283,2011-08-11,10:27:20.087,0.000,UDP,INT,0,1,64,1,147.32.84.165,1025,147.32.80.9,53,Botnet
541362,2011-08-11,10:27:22.334,0.000,UDP,INT,0,1,87,1,147.32.84.165,1025,147.32.80.9,53,Botnet
541377,2011-08-11,10:27:22.355,0.045,TCP,SRPA_,0,4,629,1,147.32.84.165,1027,74.125.232.198,80,Botnet
702906,2011-08-11,10:32:25.092,0.000,UDP,INT,0,1,78,1,147.32.84.165,1025,147.32.80.9,53,Botnet
703012,2011-08-11,10:32:25.294,0.126,TCP,SPA_,0,6,471,1,147.32.84.165,1039,195.113.232.98,80,Botnet


In [83]:
ground_truth = inf.groupby(by='Dst_IP_Addr').size()
ground_truth = ground_truth.sort_values(ascending=False)
ground_truth.head(10) 


Dst_IP_Addr
193.23.181.44      6167
174.128.246.102    3427
174.37.196.55      3357
67.19.72.206       3135
72.20.15.61        2966
173.236.31.226     1708
184.154.89.154     1678
46.4.36.120        1627
147.32.80.9         787
217.163.21.37       701
dtype: int64

In [85]:
k=[10,50,100,500,1000,5000,10000,15000,20000]


In [61]:
def sample(data, k):
    reservoir = []
    for t, item in enumerate(data['Dst_IP_Addr']):
        if t < k:
            reservoir.append(item)
        else:
            m = random.randint(0,1)
            if m > k/t:
             # random replace n item 
                ind = np.random.randint(0,k)
                reservoir[ind] = item
    return reservoir

In [73]:
s1 = sample(inf, 100)
count= collections.Counter(s1).most_common(10)
s= pd.DataFrame(count)
s.head(10)



,0,1
0,184.154.89.154,11
1,98.139.175.225,8
2,209.85.143.27,8
3,98.137.54.237,7
4,46.4.36.120,7
5,217.163.21.39,6
6,184.82.147.252,4
7,205.188.146.194,3
8,217.163.21.37,2
9,64.12.138.161,2


In [76]:
s2 = sample(inf, 1000)
count1= collections.Counter(s2).most_common(10)
s2= pd.DataFrame(count1)
s2.head(10)


,0,1
0,184.154.89.154,83
1,46.4.36.120,48
2,98.137.54.237,36
3,98.139.175.225,30
4,217.163.21.39,26
5,184.82.148.44,26
6,67.195.168.31,24
7,209.85.143.27,24
8,212.117.171.138,23
9,74.6.136.244,22


In [77]:
s2 = sample(inf, 10000)
count1= collections.Counter(s2).most_common(10)
s2= pd.DataFrame(count1)
s2.head(10)


,0,1
0,193.23.181.44,911
1,184.154.89.154,520
2,174.128.246.102,490
3,174.37.196.55,484
4,67.19.72.206,465
5,46.4.36.120,437
6,173.236.31.226,423
7,72.20.15.61,422
8,147.32.80.9,206
9,74.6.136.244,184


In [86]:
start_time = time.time()
for i in k:
    s = sample(inf, i)
    count= collections.Counter(s).most_common(10)
    s1= pd.DataFrame(count)
      
    print('For k:', i)
    print('')
    print(s1.head(10))
    print('')
dur = time.time() - start_time
print("Duration in seconds: ", dur)

For k: 10

                0  1
0   217.163.21.39  3
1   217.163.21.36  2
2   217.163.21.35  1
3    66.94.237.64  1
4  184.154.89.154  1
5  98.139.175.225  1
6   173.241.240.4  1

For k: 50

                0  1
0   217.163.21.39  7
1  184.154.89.154  7
2  98.139.175.225  6
3   64.12.138.161  2
4     46.4.36.120  2
5   98.137.54.237  2
6   217.163.21.34  2
7     65.55.37.88  1
8  216.163.188.58  1
9    74.125.39.27  1

For k: 100

                0  1
0   217.163.21.39  9
1  184.154.89.154  9
2     46.4.36.120  7
3   64.12.138.161  6
4  184.82.147.252  5
5   205.188.103.1  4
6  173.192.170.88  4
7  98.139.175.225  4
8   209.85.143.27  3
9    76.96.30.116  3

For k: 500

                0   1
0  184.154.89.154  38
1     46.4.36.120  28
2   217.163.21.39  24
3   184.82.148.43  21
4   98.137.54.237  20
5  98.139.175.225  19
6   209.85.143.27  17
7     147.32.80.9  11
8   205.188.103.2  10
9   94.100.28.114  10

For k: 1000

                 0   1
0   184.154.89.154  74
1      46.4.36.120 